In [2]:
import pickle
import pandas as pd
from pathlib import Path

In [ ]:
year = 2022
month = 2

In [4]:

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\homeworks\data\yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
y_pred.std()

5.28142955399685

In [22]:
y_pred.mean()

12.513424582912245

In [11]:

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
df['prediction'] = y_pred

In [16]:
df_result = df[['ride_id', 'prediction']]

In [20]:
output_file = fr'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\homeworks\output\{year}\{month}\result.parquet'
Path(output_file).parent.mkdir(parents=True, exist_ok=True)

In [21]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [31]:
print(fr'/homeworks/data/yellow_tripdata_{year}-{month:02}.parquet')

/homeworks/data/yellow_tripdata_2022-02.parquet
